In [26]:
from mrjob.job import MRJob
from mrjob.protocol import TextProtocol
import re

WORD_RE = re.compile(r"[\w']+")


class MRWordFreqCount(MRJob):
    OUTPUT_PROTOCOL = TextProtocol
    
    def mapper(self, _, line):                
        for word in WORD_RE.findall(line):           
            word = word.lower()
            self.increment_counter("group", "first_letter_" + word[0], 1)
            yield (word, 1)

    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, str(sum(counts))
        

In [6]:
from functools import reduce
from typing import Iterable, Tuple

lst = ["hello world", "привет мир", "hello hello world"]

# "hello" -> 3
# "world" -> 2

def our_map(k: str, v: str) -> Iterable[Tuple[str, int]]:
    words = v.split(" ")

    for word in words:  
        yield (word, 1)


def our_reduce(k: str, values: Iterable[int]) -> Iterable[Tuple[str, int]]:
    s = 0
    for v in values:
        s += v
    yield (k, s) 



list(our_map("", "hello world"))
list(our_reduce("world", [1, 1, 1]))


[('world', 3)]

# Hadoop

Начиная с версии 2.0 `Hadoop` - обычное `YARN`-приложение. В контейнерах запускают задачи для выполнения операций *map* и *reduce*.

### Основные особенности

- парадигма MapReduce
- системы мониторинга за состоянием задач
- не требует использования сложных механизмов синхронизации
- Data Locality

![MapReduce](../img/mr_diag.svg)

### Терминология

**Job** - запуск MapReduce на конктретной задачи

**Task** - запуск какой-то подзачади (map или reduce) на каком-то определенном куске данных

**Task Attempt** - попытка запуска **Task**. Hadoop может запускать одну задачу на выполнение несколько раз. Это происходит если предыдущая попытка не увенчалась успехом или медленно работает (speculative execution).

#### InputFormat

`InputFormat` - то, как должны интерпретироваться данные во входном файле, перед тем, как они попадут в Mapper, которому нужны пары ключ-значения. По умолчанию используется `TextInputFormat`, который интерпретирует входной файл как текстовый файл.

#### OutputFormat

`Outputformat` - то, как пары ключ-значения после работы Reducer'a должны записываться на `HDFS` (на самом деле не обязательно на `HDFS`).  По умолчанию используется `TextOutputFormat`, который записывает пары ключ-значения в текстовом виде. 